# LLM and Chains

Requires installation of  

pip install -U langchain-openai



In [1]:
# !pip install -U langchain

# !pip install openai
# !pip install -U langchain-openai

# !pip install -U openai

# !pip install -U langchain-openai

## Setup the environment variable for API access

#### NOTE
You MUST change the path to your own environment file

In [2]:
from dotenv import load_dotenv
import os

import warnings

warnings.filterwarnings("ignore")

# Load the file that contains the API keys
load_dotenv('C:\\Users\\raj\\.jupyter\\.env')

True

## 1. LLM : HuggingFaceHub

This will use the model hosting on the Hugging Face portal

https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.

Class

https://huggingface.co/docs/huggingface_hub/v0.20.2/en/package_reference/inference_client#huggingface_hub.InferenceClient

Supported tasks

https://huggingface.co/docs/huggingface_hub/guides/inference#supported-tasks

NOTE:

Sometimes API calls fail due to heavy usage of the model on HF. If you get a invocation error, try a again/few times.

In [9]:
from langchain.llms import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint

# model_id = 'google/flan-t5-xxl'
model_id = 'mistralai/Mistral-7B-v0.1'

prompt = "In what year was the first first Star Wars movie released?"

llm_hf = HuggingFaceHub(
    repo_id=model_id, 
    model_kwargs={ "temperature": "0.9" }
)

llm_hf = HuggingFaceEndpoint(
    repo_id=model_id, 
    # model_kwargs={ "temperature": "0.9" }
)

print(llm_hf.invoke(prompt))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\raj\.cache\huggingface\token
Login successful


Answer: 1977

A Star Wars saga is a series of films in the Star Wars franchise. It began with the release of the original trilogy, comprising the films Star Wars (1977), The Empire Strikes Back (1980), and Return of the Jedi (1983). The prequel trilogy, comprising the films The Phantom Menace (1999), Attack of the Clones (2002), and Revenge of the Sith (2005), was released between 1999 and 2005. The sequel trilogy, consisting of The Force Awakens (2015), The Last Jedi (2017), and The Rise of Skywalker (2019), was released from 2015 to 2019. The anthology films Rogue One (2016) and Solo (2018) were released in 2016 and 2018, respectively.

The first Star Wars movie was released in 1977.


## 2. LLMChain

Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step

https://python.langchain.com/docs/modules/chains

https://api.python.langchain.com/en/stable/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain

https://api.python.langchain.com/en/latest/llms/langchain_community.llms.huggingface_endpoint.HuggingFaceEndpoint.html

In [10]:
from langchain.prompts import PromptTemplate

superbowl_template = "who won super bowl in year {year}?"

prompt_template = PromptTemplate(
    template = superbowl_template,
    input_variables = ["year"]
)

In [11]:
from langchain import LLMChain

# Create a simple chain
chain = LLMChain(
            prompt = prompt_template, 
            llm = llm_hf,
            output_key = "superbowl_winner"
        )

In [12]:
# Both the lines will yield the same result

response = chain.invoke(1977)    

# response = chain.invoke({"year": 1977}))

In [13]:
response 

{'year': 1977,
 'superbowl_winner': '\n\nLos Angeles Rams\n\nWho won Super Bowl 1977?\n\nPittsburgh Steelers\n\nWhat year was the Super Bowl 1977?\n\nSuper Bowl X\n\nWho won Super Bowl 1978?\n\nPittsburgh Steelers\n\n## Who won the Super Bowl in 1977?\n\nLos Angeles Rams\n\n## Who won the Super Bowl in 1979?\n\nPittsburgh Steelers\n\n## Who won the Super Bowl in 1978?\n\nSuper Bowl XIII (XIII) was an American football game played on January 21, 1979 at the Orange Bowl in Miami, Florida to decide the National Football League (NFL) champion following the 1978 NFL season.\n\n## Who won the Super Bowl in 1977?\n\nThe Los Angeles Rams won their first and only Super Bowl victory on January 15, 1979, beating the Pittsburgh Steelers 31–19 in Super Bowl XIV.\n\n## Who won the Super Bowl in 1979?\n\nPittsburgh Steelers\n\n## Who won the Super Bowl in 1980?\n\nPittsburgh Steelers\n\n## Who won Super Bowl 1978?\n\nPittsburgh Steelers\n\n## Who won the Super Bowl in 1976?\n\nThe 1976 Super Bowl was

## 3. Simple LLM Sequential Chain

Use the HuggingFace model google/flan-t5-xxl to answer a question. The send the question/answer pair to OpenAI/gpt3.5 to verify if the answer is correct or not.

https://api.python.langchain.com/en/stable/chains/langchain.chains.sequential.SimpleSequentialChain.html#langchain.chains.sequential.SimpleSequentialChain

### Create a LLMChain with HF model

In [14]:
# Template for HF model
prompt_template_1 = PromptTemplate.from_template(
    template="define {term}", 
    inputs=['term']
)

# Create the LLM chain object
llm_chain_1 = LLMChain(
    llm = llm_hf,
    prompt = prompt_template_1,
    output_key = "definition"
)

## example
# llm_chain_1.invoke("momentum")

### Create the OpenAI LLM

LLM

https://python.langchain.com/docs/integrations/platforms/openai

In [15]:
from langchain_openai import OpenAI

# default "gpt-3.5-turbo"
llm_openai = OpenAI()

In [16]:
# Template for OpenAI
prompt_template_2 = PromptTemplate.from_template(
    template="""rate the accuracy of the definition on a scale of (0 to 5).
                Definition of :  {definition}""", 
    inputs=['definition']
)

## The following template if used with SimpleSequentialChain will FAIL
## Added for demonstration purposes only - to try uncomment the code below
# prompt_template_openai =PromptTemplate.from_template(template="""
# rate the accuracy of the definition on a scale of (0 to 5).
# Definition of {term}:  {definition}""", inputs=['definition','term'])


llm_chain_2 = LLMChain(
    llm = llm_openai,
    prompt = prompt_template_2,
    output_key = "score"
)

In [17]:
from langchain.chains import SimpleSequentialChain

simple_sequential_chain = SimpleSequentialChain(
    chains=[llm_chain_1, llm_chain_2],
    verbose = True,
)

In [18]:
# Input = String value if there is ONLY one input variable
# Input = Dictionary if there are multiple input variables

response = simple_sequential_chain.invoke("moment of interia")



> Entering new SimpleSequentialChain chain...
 to be m = I/r^2

where I is the moment of inertia of the object and r is the radius.

The moment of inertia of an object is the inertia of the object times its distance from the rotation axis.

For example, consider a sphere. The moment of inertia of a sphere is:

I = (2/3)mR^2

where m is the mass of the sphere and R is the radius.

The moment of inertia of a cube is:

I = (2/3)mR^2

where m is the mass of the cube and R is the radius.

The moment of inertia of a cylinder is:

I = (2/3)mR^2

where m is the mass of the cylinder and R is the radius.

The moment of inertia of a sphere is:

I = (2/3)mR^2

where m is the mass of the sphere and R is the radius.

The moment of inertia of a cube is:

I = (2/3)mR^2

where m is the mass of the cube and R is the radius.

The moment of inertia of a cylinder is:

I = (2/3)mR^2

where m is the mass of the cylinder and R is the radius.

The moment of inertia of a sphere is:

I = (2/3)mR^2

where m is 

In [ ]:
print(response)